In [1]:
!pip install uproot awkward 
from uproot_io import Events, View
import numpy as np
import matplotlib.pyplot as plt
import features as f

cheated_0 = Events('CheatedRecoFile_0.root')

  Using cached uproot-5.5.1-py3-none-any.whl.metadata (33 kB)
  Using cached awkward-2.7.1-py3-none-any.whl.metadata (7.0 kB)
  Using cached cramjam-2.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.9 kB)
  Using cached awkward_cpp-42-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached uproot-5.5.1-py3-none-any.whl (362 kB)
Using cached awkward-2.7.1-py3-none-any.whl (863 kB)
Using cached awkward_cpp-42-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (637 kB)
Using cached cramjam-2.9.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)


In [2]:
from statsmodels.nonparametric.smoothers_lowess import lowess

def LOWESS_standard_deviation(events, event_idx):
    w_hits = events.reco_hits_w[event_idx]
    adc_values = events.reco_adcs_w[event_idx]
    if len(w_hits) <= 5:
        return print('Less than 5 hits in event, not sufficient to plot.')
    
    frac = 0.2  # Fraction of data to use for smoothing
    smoothed = lowess(adc_values, w_hits, frac=frac)
    adc_lowess = smoothed[:, 1]

    # Step 1: Calculate the mean
    adc_mean = np.mean(adc_lowess)
    
    # Step 2: Compute squared differences
    squared_differences = (adc_lowess - adc_mean) ** 2
    
    # Step 3: Calculate standard deviation
    std_dev = np.sqrt(np.mean(squared_differences))  # Population

    return std_dev

In [3]:
sd = LOWESS_standard_deviation(cheated_0, 1554)

print(sd)

2.1182415979666174


In [5]:
def q4_ratio(events, event_idx):
    adcs = events.reco_adcs_w[event_idx]

    q4_idx = len(adcs) // 4

    adcs_q4 = adcs[-q4_idx:]

    ratio = sum(adcs_q4) / sum(adcs)

    return ratio

In [6]:
q4r = adc_q4_ratio(cheated_0, 1554)

print(q4r)

0.2490491619608897


In [11]:
def line(events, event_idx):
    w_hits = np.array(events.reco_hits_w[event_idx])
    x_hits = np.array(events.reco_hits_x_w[event_idx])
    
    # Calculate differences between consecutive points
    dx = np.diff(w_hits)
    dy = np.diff(x_hits)
    
    # Compute segment lengths
    segment_lengths = np.sqrt(dx**2 + dy**2)
    
    # Total arc length (line integral)
    total_length = np.sum(segment_lengths)
    
    # Normalize by the number of points
    normalised_length = total_length / len(w_hits)

    return normalised_length

In [12]:
arc_l = line(cheated_0, 1554)

print(arc_l)

0.6159520827900441
